In [8]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
import os


def get_players_and_links(search_link, pages):
    
    players = []
    links = []
        
    for page in range(1, pages+1):
        link2 = ''.join([search_link,str(page)])
        soup = BeautifulSoup(requests.get(link2).content, 'lxml')

        table = soup.find('table', attrs={'class':'listing'})
        table_rows = table.find_all('tr')

        
        for tr in table_rows[1:]:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            players.append(row)

        for link in soup.findAll('a', attrs={'href': re.compile("19/players/prices")}):
                            links.append(link.get('href'))

    [print(p[1].strip()) for p in players]
    [print(l) for l in links]
    #assert len(links)==len(players)
    
    return players, links


class PlayerItem:
    def __init__(self, item, link):
        #item = l[1]
        self.name = item[1].split('\n ')[1].strip()
        self.program = item[1].split('\n ')[-1].strip()
        self.ovr = int(item[2])
        self.pos = item[3]
        self.date = [' ']
        self.link = 'https://www.muthead.com' + link
        
    def string(self):
        return f"Player: {self.name}\nProgram: {self.program}\nOVR: {self.ovr}\nDate: {self.date}"
    
    def as_dataframe(self):
        return pd.DataFrame([{'name': self.name, 'program': self.program, 'date': self.date, 
                              'link': self.link, 'ovr': self.ovr, 'pos': self.pos}])
    
def get_release_date(link):
    soup = BeautifulSoup(requests.get(link).content, 'lxml')
    try:
        date = soup.find_all('script', type="text/javascript")[1].string.split('series:')[1].split('Date.UTC')[1].split(')')[0].lstrip('()').split(', ')
        date = [int(x) for x in date]
    except:
        date = soup.find_all('script', type="text/javascript")
        print('(exception)')
    # year, month, day = date
    return date


def get_and_save_player_dates(goal_program, date):
    global pages_map
    global link_map
    
    n_pages = pages_map[goal_program]
    link = f'https://www.muthead.com/19/players?filter-market=4&filter-program-19={link_map[goal_program]}&page='
    print(link)
    players, links = get_players_and_links(link, n_pages)
    #print(players)
    # create list of PlayerItem objects to store data including dates
    player_items = [PlayerItem(item=player, link=link_) for player, link_ in zip(players, links)]#[:5]
    
    #if os.path.isfile(f'{date}_{goal_program}.csv'):
        #df = pd.read_csv(f'{date}_{goal_program}.csv', index_col=0)
        #print('reading previous file... (not doing anything yet)')
        
    # TODO: put full filtering here to limit unneccessary scraping
    # Convert to dataframe and save as csv
    df = pd.concat([p.as_dataframe() for p in player_items])
    df.reset_index(drop=True, inplace=True)

    df['date_dtype'] = df['date'].apply(lambda x: type(x[0])).astype(str)
    df['dated'] = df['date_dtype'].apply(lambda x: x =="<class 'int'>")*1
    
    df['full'] = 0
    indices = [idx[0] for name, idx in df.groupby(['name'])['ovr'].groups.items()]
    df['index'] = df.index
    df['full'] = df['index'].apply(lambda x: 1 if x in indices else 0)
    
    dates = []
    # Fetch release dates of players
    for link in df['link'].values:
        time.sleep(5)
        date = get_release_date(link)
        dates.append(date)
    df['date'] = dates
    df.to_csv(f'{date}_{goal_program}.csv')
    
    return df

In [ ]:
edelman retr

In [9]:
# Pages of players, TODO: lazy! make it better
pages_map = {'L': 16, 'UL': 9, 'Ghosts': 1, 'SBP': 1, 'Gauntlet': 1} 
# Legend: 227, UL: 265
link_map = {'L': 227, 'UL': 265, 'Ghosts': 256, 'SBP': 262, 'Gauntlet': 224}

# INPUTS
goal_program = 'Gauntlet'
date = 'apr28'

df = get_and_save_player_dates(goal_program=goal_program, date=date)

https://www.muthead.com/19/players?filter-market=4&filter-program-19=224&page=
Devonta Freeman
        
    
The Gauntlet
Donovan McNabb
        
    
The Gauntlet
John Randle
        
    
The Gauntlet
Richard Sherman
        
    
The Gauntlet
Roquan Smith
        
    
The Gauntlet
Ted Hendricks
        
    
The Gauntlet
Steve Hutchinson
        
    
The Gauntlet
Lorenzo Neal
        
    
The Gauntlet
Willie Roaf
        
    
The Gauntlet
Sean Taylor
        
    
The Gauntlet
Cameron Heyward
        
    
The Gauntlet
Jason Kelce
        
    
The Gauntlet
Lamar Miller
        
    
The Gauntlet
Brian Orakpo
        
    
The Gauntlet
Rodger Saffold III
        
    
The Gauntlet
Jaire Alexander
        
    
The Gauntlet
Rashaan Evans
        
    
The Gauntlet
Baker Mayfield
        
    
The Gauntlet
D.J. Moore
        
    
The Gauntlet
Vita Vea
        
    
The Gauntlet
/19/players/prices/60137-devonta-freeman
/19/players/prices/60139-donovan-mcnabb
/19/players/prices/601

In [11]:
df#.head()

,date,link,name,ovr,pos,program,date_dtype,dated,full,index
0,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/6013...,Devonta Freeman,95,HB,The Gauntlet,<class 'str'>,0,1,0
1,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/6013...,Donovan McNabb,95,QB,The Gauntlet,<class 'str'>,0,1,1
2,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/6014...,John Randle,95,DT,The Gauntlet,<class 'str'>,0,1,2
3,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/6013...,Richard Sherman,95,CB,The Gauntlet,<class 'str'>,0,1,3
4,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/6013...,Roquan Smith,95,MLB,The Gauntlet,<class 'str'>,0,1,4


In [29]:
def make_entry(date, player_id, name, ovr, pos, program, dated):
    entry = {'date': date, 
             'link': f'https://www.muthead.com/19/players/prices/{player_id}',
             'name': name,
             'ovr': ovr, 
             'pos': pos,
             'program': program,
             'dated': dated}
    dataframe = pd.DataFrame(entry, index=[0])
    #columns = dataframe.columns
    #assert len(columns) == len(entries)
    return dataframe

make_entry([0], 999, 'bob', 98, 'rb', 'test', 0)

,date,link,name,ovr,pos,program,dated
0,0,https://www.muthead.com/19/players/prices/999,bob,98,rb,test,0


In [15]:
full = df[ df['full']==1]
full['dated'].value_counts()

0    5
Name: dated, dtype: int64

In [21]:
full.drop(['full', 'index', 'date_dtype'], axis=1, inplace=True)
full = full.loc[1:3]

In [23]:
full = full.iloc[:2]

In [24]:
full

,date,link,name,ovr,pos,program,dated
1,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/6013...,Donovan McNabb,95,QB,The Gauntlet,0
2,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/6014...,John Randle,95,DT,The Gauntlet,0


In [30]:
ted = make_entry(date='[2018, 11, 14]', 
                 player_id=60119,
                 name='Ted Hendricks',
                 ovr=93,
                 pos='LOLB',
                 program='The Gauntlet',
                 dated=1)
steve = make_entry(date='[2018, 11, 14]', 
                 player_id=60118,
                 name='Steve Hutchinson',
                 ovr=93,
                 pos='LG',
                 program='The Gauntlet',
                 dated=1)
neal = make_entry(date='[2018, 11, 14]', 
                 player_id=60116,
                 name='Lorenzo Neal',
                 ovr=93,
                 pos='FB',
                 program='The Gauntlet',
                 dated=1)
roaf = make_entry(date='[2018, 11, 14]', 
                 player_id=60117,
                 name='Willie Roaf',
                 ovr=93,
                 pos='LT',
                 program='The Gauntlet',
                 dated=1)
st = make_entry(date='[2018, 11, 14]', 
                 player_id=60120,
                 name='Sean Taylor',
                 ovr=93,
                 pos='FS',
                 program='The Gauntlet',
                 dated=1)

entries = [ted, steve, neal, roaf, st]
entry_df = pd.concat(entries)

In [31]:
entry_df

,date,link,name,ovr,pos,program,dated
0,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/60119,Ted Hendricks,93,LOLB,The Gauntlet,1
0,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/60118,Steve Hutchinson,93,LG,The Gauntlet,1
0,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/60116,Lorenzo Neal,93,FB,The Gauntlet,1
0,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/60117,Willie Roaf,93,LT,The Gauntlet,1
0,"[2018, 11, 14]",https://www.muthead.com/19/players/prices/60120,Sean Taylor,93,FS,The Gauntlet,1


In [33]:
full = pd.concat([full, entry_df])

In [35]:
full.reset_index(drop=True, inplace=True)
full.to_csv('april28_full_gauntlet.csv')

In [36]:
# NIGHT GUARDIAN
mjg = make_entry(date='[2018, 9, 25]', 
                 player_id=51534,
                 name='Mean Joe Greene',
                 ovr=94,
                 pos='DT',
                 program='Night Guardians',
                 dated=1)
ntl = make_entry(date='[2018, 9, 26]', 
                 player_id=52216,
                 name='Night Train Lane',
                 ovr=94,
                 pos='CB',
                 program='Night Guardians',
                 dated=1)
ngs = pd.concat([mjg, ntl])

In [38]:
ngs.reset_index(drop=True, inplace=True)
ngs.to_csv('april28_full_nightguardians.csv')

In [39]:
# HARVEST
lt = make_entry(date='[2018, 10, 15]', 
                 player_id=61002,
                 name='Lawrence Taylor',
                 ovr=95,
                 pos='ROLB',
                 program='Harvest',
                 dated=1)
wp = make_entry(date='[2018, 10, 15]', 
                 player_id=61001,
                 name='Walter Payton',
                 ovr=95,
                 pos='RB',
                 program='Harvest',
                 dated=1)
harvest = pd.concat([lt, wp])

harvest.reset_index(drop=True, inplace=True)
harvest.to_csv('april28_full_harvest.csv')

In [40]:
# PLAYOFFS
wm = make_entry(date='[2019, 0, 5]', 
                 player_id=62502,
                 name='Willie McGinest',
                 ovr=97,
                 pos='ROLB',
                 program='NFL Playoffs',
                 dated=1)
sss = make_entry(date='[2019, 0, 5]', 
                 player_id=62501,
                 name='Steve Smith Sr',
                 ovr=97,
                 pos='WR',
                 program='NFL Playoffs',
                 dated=1)
playoffs = pd.concat([wm, sss])

playoffs.reset_index(drop=True, inplace=True)
playoffs.to_csv('april28_full_playoffs.csv')

In [42]:
# Assorted: mutmonster, combine, draft?
calvin = make_entry(date='[2018, 9, 5]', 
                 player_id=51501,
                 name='Calvin Johnson',
                 ovr=94,
                 pos='WR',
                 program='MUT Monsters',
                 dated=1)

cj2k = make_entry(date='[2019, 1, 28]', 
                 player_id=65601,
                 name='Chris Johnson',
                 ovr=99,
                 pos='RB',
                 program='NFL Combine',
                 dated=1)

lott = make_entry(date='[2018, 7, 15]', 
                 player_id=55100,
                 name='Ronnie Lott',
                 ovr=94,
                 pos='FS',
                 program='Team Diamonds',
                 dated=1)
# Draft
kj = make_entry(date='[2019, 2, 30]', 
                 player_id=66113,
                 name='Keyshawn Johnson',
                 ovr=99,
                 pos='WR',
                 program='NFL Draft',
                 dated=1)
cb = make_entry(date='[2019, 3, 3]', 
                 player_id=66138,
                 name='Chuck Bednarik',
                 ovr=99,
                 pos='LOLB',
                 program='NFL Draft',
                 dated=1)
bs = make_entry(date='[2019, 2, 28]', 
                 player_id=66101,
                 name='Bruce Smith',
                 ovr=99,
                 pos='RE',
                 program='NFL Draft',
                 dated=1)
assorted = pd.concat([calvin, cj2k, lott, kj, cb, bs])

assorted.reset_index(drop=True, inplace=True)
assorted.to_csv('april28_full_assorted.csv')

In [43]:
# Snow
mawae = make_entry(date='[2018, 11, 9]', 
                 player_id=10201100,
                 name='Kevin Mawae',
                 ovr=94,
                 pos='C',
                 program='Snow Beasts',
                 dated=1)
long = make_entry(date='[2018, 11, 9]', 
                 player_id=10202911,
                 name='Howie Long',
                 ovr=94,
                 pos='RE',
                 program='Snow Beasts',
                 dated=1)
steve = make_entry(date='[2018, 11, 9]', 
                 player_id=10200200,
                 name='Steve Atwater',
                 ovr=94,
                 pos='SS',
                 program='Snow Beasts',
                 dated=1)
herm = make_entry(date='[2018, 11, 9]', 
                 player_id=10202818,
                 name='Herman Moore',
                 ovr=94,
                 pos='WR',
                 program='Snow Beasts',
                 dated=1)

vick = make_entry(date='[2019, 0, 8]', 
                 player_id=62251,
                 name='Michael Vick',
                 ovr=96,
                 pos='QB',
                 program='Ghosts of Madden',
                 dated=1)
champ = make_entry(date='[2019, 0, 8]', 
                 player_id=62271,
                 name='Champ Bailey',
                 ovr=96,
                 pos='CB',
                 program='Ghosts of Madden',
                 dated=1)
tony = make_entry(date='[2019, 0, 8]', 
                 player_id=62261,
                 name='Tony Gonzalez',
                 ovr=96,
                 pos='TE',
                 program='Ghosts of Madden',
                 dated=1)

snow = pd.concat([mawae, long, steve, herm, vick, champ, tony])

snow.reset_index(drop=True, inplace=True)
snow.to_csv('april28_full_snow.csv')